# La QFT y sumadores cuánticos



Mario Quiñones Pérez

## tKet

En esta práctica profundizaremos en la Transformada Cuántica de Fourier (QFT), que es fundamental para aplicar la phase kickback. Además, aprovecharemos el diseño de este componente para implementar un sumador cuántico.

In [1]:
from pytket import Circuit
from pytket.extensions.qiskit import AerBackend
import numpy as np

### QFT

La Transformada Cuántica de Fourier (QFT) es un método para trabajar en el dominio de la fase, esto nos permite aprovechar el fenómeno de la superposición, visto en la práctica anterior. Con esta puerta pasamos del dominio de la amplitud al dominio de la fase para poder así influir en la salida al modificar las amplitudes antes de realizar la medición favoreciendo así aquella salida que nos interesa, en esta práctica la de la suma.

En primer lugar hay que aplicar una puerta Hadamard a todos los qubits y, a continuación, hay que aplicar rotaciones controladas. Dichas rotaciones siguen la siguiente regla: el qubit xi controla una rotación Rπ/2^(i−j) sobre el qubit objetivo xi−j.

In [2]:
##QFT

# Esta función crea una transformada cuántica de Fourier para un número de qubits dado,
# si no se especifica se creará para dos qubits
def QFT(qc, qbits = 2):
    qbits2 = qbits - 1
    
    for i in range(qbits):
        # Por cada qubit se aplica una puerta hadamard
        qc.H(i)
        # Y se calcula en control en qué posición se empieza el control de rotaciones 
        control = i + 1
        
        # Se realizan dichas rotaciones , tantas como qbits2 diga
        for j in range(qbits2):
            qc.CRz(np.pi/np.power(2, (j+1)), control, i)
            control = control + 1
            
        # Se añade entre grupos de rotaciones por claridad
        if (i < qbits - 1):
            c.add_barrier([0,1,2,3])
        # Se calcula cuantas rotaciones tendrá el siguiente qubit (una menos que la anterior)
        qbits2 = qbits2 - 1

En esta práctica y para la representación de los circuitos en tket se ha utilizado tanto la función to_latex_file('name.tex') como la aplicación TexWorks para la creación de pdfs con las imágenes de los circuitos que luego se añadirán al código como png. Se adjuntaran los png devueltos para la correcta visualización de los notebook.

In [3]:
c = Circuit(4,4)
QFT(c, qbits = 4)

# Función utilizada para recuperar el circuito como archivo latex, para pasarlo pdf y por ultimo a png.
c.to_latex_file('QFT.tex')

![1.png](1.png)

### AQFT

Como se ha mencionado anteriormente, la QFT requiere realizar rotaciones controladas. El ángulo de rotación puede llegar a ser muy pequeño si tenemos muchos qubits de entrada, por lo que controlar físicamente dichas rotaciones con precisión absoluta es muy complicado.

Debido a que realmente estas rotaciones tan pequeñas no contribuyen en gran medida al resultado final, se puede utilizar la QFT aproximada (AQFT). Con la que se permite un número máximo de rotaciones controladas, por ejemplo 2, como aparece en la función AQFT.

In [4]:
##AQFT

# Esta función crea una transformada cuántica de Fourier aproximada para un número de qubits dado y
# con una cantidad máxima de rotaciones 'maxc', si no se especifica se creará para dos qubits y con maxc = 2
def AQFT(qc, qbits = 2, maxc = 2):
    qbits2 = qbits - 1
    
    for i in range(qbits):
        qc.H(i)
        control = i + 1
        
        for j in range(qbits2):
            # Solo se hacen más rotaciones controladas si no nos pasamos del máximo dado
            if(j < maxc):
                qc.CRz(np.pi/np.power(2, (j+1)), control, i)
                control = control + 1
        qbits2 = qbits2 - 1


In [5]:
c = Circuit(4,4)
AQFT(c, qbits = 4)

# Función utilizada para recuperar el circuito como archivo tex para pasarlo pdf y luego a png
c.to_latex_file('AQFT.tex')

![2.png](2.png)

### Sumador Cuántico

Dados dos operandos de entrada A y B, de n qubits cada uno, la manera de realizar dicha operación es conservar uno de los dos, en este caso B y en el otro acumular los qubits para que el resultado sea A + B, así dicho circuito es reversible ya que tenemos B para aplicar la resta y recuperar A.

Este sumador de Draper tiene tres fases:

    - Aplicar la QFT o AQFT a A para pasar al dominio de la fase y así con el sumador y
    sus rotaciones controladas poder influir en la salida del circuito

    - La suma en el dominio de la fase. Realizada mediante rotaciones controladas, 
    siguiendo la regla en la que Bj aplica una rotación controlada al qubit Ai con 
    una rotación de π/(2^j)

    - La QFT inversa (IQFT) al resultado de la suma que es lo mismo que aplicar la 
    QFT pero las puertas en el orden contrario a esta. Se utiliza para pasar de nuevo 
    al dominio de la amplitud y medir los resultados obtenidos del sumador que modifica 
    la fase.

In [6]:
##adder

# Esta se realiza mediante rotaciones controladas a dos elementos con el mismo
# número de qubits A y B siguiendo la regla en la que Bj aplica una rotación
# controlada al qubit Ai con una rotación de π/(2^j)


# Esta función crea un sumador para un número de qubits dado, si no se especifica 
# se creará para un sumador de dos qubits
def adder(qc, qbits = 2):
    qbits2 = qbits - 1
    
    for i in range(qbits):
        # Se calcula en control en que qubit empezará el control de los qubits
        control = i + qbits
        # Se aplica la rotación Z a cada qubit de A con el que está en su misma posición en B 
        qc.CZ(control,i)
        
        # Se realizan el resto de rotaciones por qubit tantas como diga qbits2
        for j in range(qbits2):
            control = control + 1
            qc.CRz(np.pi/np.power(2, (j+1)), control, i)
            
        # Se añade una barrera al final de cada grupo de rotaciones por claridad
        if (i < qbits - 1):
            c.add_barrier([0,1,2,3,4,5,6,7])
        # Se calcula cuantas rotaciones tendrá el siguiente qubit (una menos que la anterior) 
        qbits2 = qbits2 - 1

In [7]:
##IQFT

# Esta función crea una transformada cuántica de Fourier inversa para un número de qubits dado, 
# si no se especifica se creará para dos qubits. Ademas añadiremos un maximo de rotaciones
# controladas 'maxc' para poder utilizar esta función con la AQFT tambien
def IQFT(qc, qbits = 2, maxc = 2):
    qbits2 = 0
    
    for i in range(qbits):
        control = qbits - 1
        
        for j in range(qbits2):
            if(j < maxc):
                qc.CRz(-np.pi/np.power(2, (j+1)), control, qbits - i - 1)
                control = control - 1
            
        qc.H(qbits - i - 1)
        if (i < qbits - 1):
            c.add_barrier([0,1,2,3])
        qbits2 = qbits2 + 1

In [8]:
c = Circuit(8,8)

c.X(1)
c.X(2)
c.X(3)
c.X(6)

QFT(c, qbits = 4)

c.add_barrier([0,1,2,3,4,5,6,7], [0,1,2,3,4,5,6,7])

adder(c, qbits = 4)

c.add_barrier([0,1,2,3,4,5,6,7], [0,1,2,3,4,5,6,7])

IQFT(c, qbits = 4, maxc = 3)

# Función utilizada para recuperar el circuito como archivo tex para pasarlo pdf y luego a png
c.to_latex_file('complete_adder.tex')

![3.png](3.png)

![4.png](4.png)

![5.png](5.png)

### Simulaciones

Se realiza la simulación de la misma suma 0111 y 1000 con la AQFT y la QFT y vemos que ambas dan siempre la solución correcta ya que la AQFT es una aproximación bastante buena da la QFT en la mayoría de situaciones a partir de un determinado número de qubits ya que las rotaciones que se harán serán muy pequeñas como para afectar a la solución 

In [9]:
c = Circuit(8,8)

c.X(1)
c.X(2)
c.X(3)
c.X(4)

QFT(c, qbits = 4)
adder(c, qbits = 4)
IQFT(c, qbits = 4)
c.measure_all()  # medir todos los qbits(1 en este caso)

b = AerBackend()                # conectar al backend
b.compile_circuit(c)            # compilar el circuito para satisfacer las condiciones del backend
handle = b.process_circuit(c, n_shots = 1000)  # ejecutar 1000 veces
counts = b.get_result(handle).get_counts()   # recuperar los resultados
print(counts)

Counter({(1, 1, 1, 1, 1, 0, 0, 0): 1000})


In [10]:
c = Circuit(8,8)

c.X(1)
c.X(2)
c.X(3)
c.X(4)

AQFT(c, qbits = 4)
adder(c, qbits = 4)
IQFT(c, qbits = 4)
c.measure_all()  # medir todos los qbits(1 en este caso)

b = AerBackend()                # conectar al backend
b.compile_circuit(c)            # compilar el circuito para satisfacer las condiciones del backend
handle = b.process_circuit(c, n_shots = 1000)  # ejecutar 1000 veces
counts = b.get_result(handle).get_counts()   # recuperar los resultados
print(counts)

Counter({(1, 1, 1, 1, 1, 0, 0, 0): 1000})
